In [47]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [52]:
rent_obj = []
all_property = []

for page in range(28,29):
    link = 'https://www.apartments.com/houses/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
#     print(soup)
#     page 1 - 2
#     results = soup.find_all('article', class_='placard placard-option has-header js-premiumplus')
#     page 2-28
    results = soup.find_all('article', class_='placard placard-option no-header js-premium')
#     page
#     results = soup.find_all('article', class_='prosumer placard')

    # Loop through returned results and get the hyperlink of the properties of the page
    print(results)
    for result in results:
#         print('result: ', result)
        find_tag = result.find('a', class_="property-link")['href']
        print(find_tag)
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
#         print(result)
        full_address = result.find('div', class_ = "property-address").text
        print(full_address)
        city = new_results.find('div', class_ = "propertyAddress").select('span')[0].text
        state = new_results.find('div', class_ = "propertyAddress").select('span')[1].text
        zip_code = new_results.find('div', class_ = "propertyAddress").select('span')[2].text
        street = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
        bedroom_container = new_results.find('div', class_="rentRollupContainer")
        house_details = new_results.find('table', class_="availabilityTable")
        bath_number = house_details.find('td', class_="baths").find('span', class_="shortText").text.replace('\r\n                    ', "").strip()
        house_square_feet = house_details.find('td', class_="sqft").text
        square_feet = house_square_feet.split(" ")[0];
        lease_length = house_details.find('td', class_="leaseLength").text
        availability_check = house_details.find('td', class_="available").text.replace('\r\n                    ', "").strip()
        all_the_rents = bedroom_container.findAll('span', class_="rentRollup")

        bedroom_info = []
        for rent_block in all_the_rents:
            bedroom_type = rent_block.find('span', class_="shortText").text
            try:
                price = re.search('\$.* ', rent_block.text)
                price = price.group(0)[:-3]

            except:
                if "Call for Rent" in rent_block.text:
                    price = ""
                else:
                    price = re.search('\$.*', rent_block.text).group()[:-1]
            bedroom_info.append({
                    'bedroom_type': bedroom_type,
                    'price': price
                })
        rent_obj = {
                "City_Rent": city,
                "Type": "Houses",
                "State_Rent": state,
                "Zip_code": zip_code,
                "Bedroom_Info": bedroom_info,
                "Full_Address": full_address,
                "Rent_link": find_tag,
                "Bath_Info": bath_number,
                "Square_Feet_Data": house_square_feet,
                "Square_Feet": square_feet,
                "Lease_length": lease_length,
                "Availability_check": availability_check
            }
        for room_type in bedroom_info:
            rent_obj[room_type["bedroom_type"]] = room_type["price"];
        
        all_property.append(rent_obj)
        time.sleep(2)
# print(all_property)

[<article class="placard placard-option no-header js-premium" data-listingid="fet1ndv" data-url="https://www.apartments.com/260-emerald-bay-laguna-beach-ca/fet1ndv/">
<section class="placard-content">
<div class="content-inner">
<div class="media-wrapper">
<div class="media-sizer"></div>
<div class="media-outer">
<div class="media-inner">
<div class="media">
<div class="imageContainer carousel slide" data-interval="false">
<div class="carouselInner">
<div class="item active" style='background-image: url("https://images1.apartments.com/i2/WyKyN6tn77pqtEtF7jXm3DRaY92p4dbnz8DN7LoxbDw/118/260-emerald-bay-laguna-beach-ca-building-photo.jpg");' title="Building Photo - 260 Emerald Bay">
</div>
</div>
</div>
</div>
</div>
</div>
</div>
<div class="property-info">
<div class="content-wrapper">
<div class="property-details-wrapper no-header no-specials">
<span class="listingFreshness">
<span class="lastUpdated">
<i class="rotateRightIcon" title="Last Updated"></i>
<span>2 wks Ago</span>
</span>


260 Emerald Bay, Laguna Beach, CA 92651
https://www.apartments.com/587-crane-blvd-los-angeles-ca/ld8xgje/
587 Crane Blvd, Los Angeles, CA 90065


KeyboardInterrupt: 

In [49]:
rent_data = pd.DataFrame(all_property)
rent_data

,City_Rent,Type,State_Rent,Zip_code,Bedroom_Info,Full_Address,Rent_link,Bath_Info,Square_Feet_Data,Square_Feet,Lease_length,Availability_check,1 Bed,2 Beds,3 Beds,4 Beds,Studio,5 Beds,6 Beds,1 – 3 Beds
0,Castro Valley,Houses,CA,94552,"[{'bedroom_type': '1 Bed', 'price': '$1,49'}]","20187 Crow Creek Rd, Castro Valley, CA 94552",https://www.apartments.com/20187-crow-creek-rd...,1 BA,250 Sq Ft,250,1 Month Lease,Available Now,"$1,49",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mountain View,Houses,CA,94041,"[{'bedroom_type': '2 Beds', 'price': '$3,09'}]","644 Church St, Mountain View, CA 94041",https://www.apartments.com/644-church-st-mount...,1 BA,"1,270 Sq Ft","1,270",6 Month Lease,Available Now,NaN,"$3,09",NaN,NaN,NaN,NaN,NaN,NaN
2,North Hollywood,Houses,CA,91606,"[{'bedroom_type': '2 Beds', 'price': '$2,80'}]","6022 Bonner Ave, North Hollywood, CA 91606",https://www.apartments.com/6022-bonner-ave-nor...,2 BAs,"1,175 Sq Ft","1,175",12 Month Lease,Available Now,NaN,"$2,80",NaN,NaN,NaN,NaN,NaN,NaN
3,Pasadena,Houses,CA,91101,"[{'bedroom_type': '3 Beds', 'price': '$2,87'}]","465 Marengo N Ave, Pasadena, CA 91101",https://www.apartments.com/465-marengo-n-ave-p...,2 BAs,"1,250 Sq Ft","1,250",12 Month Lease,Available Now,NaN,NaN,"$2,87",NaN,NaN,NaN,NaN,NaN
4,Los Angeles,Houses,CA,91040,"[{'bedroom_type': '3 Beds', 'price': '$2,70'}]","10525 Greenburn Ave, Los Angeles, CA 91040",https://www.apartments.com/10525-greenburn-ave...,2 BAs,"1,000 Sq Ft","1,000",12 Month Lease,Available Now,NaN,NaN,"$2,70",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Los Angeles,Houses,CA,90063,"[{'bedroom_type': '2 Beds', 'price': '$2,50'}]","3117 2nd E St, Los Angeles, CA 90063",https://www.apartments.com/3117-2nd-e-st-los-a...,1 BA,900 Sq Ft,900,12 Month Lease,Available Now,NaN,"$2,50",NaN,NaN,NaN,NaN,NaN,NaN
196,Oakland,Houses,CA,94605,"[{'bedroom_type': '2 Beds', 'price': '$3,50'}]","11260 Kerrigan Dr, Oakland, CA 94605",https://www.apartments.com/11260-kerrigan-dr-o...,2½ BAs,"1,700 Sq Ft","1,700",6 Month Lease,Available Now,NaN,"$3,50",NaN,NaN,NaN,NaN,NaN,NaN
197,Glendale,Houses,CA,91206,"[{'bedroom_type': '3 Beds', 'price': '$4,50'}]","116 Coventry Pl, Glendale, CA 91206",https://www.apartments.com/116-coventry-pl-gle...,3½ BAs,"2,100 Sq Ft","2,100",12 Month Lease,Available Now,NaN,NaN,"$4,50",NaN,NaN,NaN,NaN,NaN
198,Los Angeles,Houses,CA,91401,"[{'bedroom_type': '3 Beds', 'price': '$4,00'}]","14347 Martha St, Los Angeles, CA 91401",https://www.apartments.com/14347-martha-st-los...,1½ BAs,"1,300 Sq Ft","1,300",,Available Now,NaN,NaN,"$4,00",NaN,NaN,NaN,NaN,NaN


In [50]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q4-2020_Apartment.com_Houses_Data_page21-28.csv", index=False, header=True)